In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import invgamma, multivariate_normal, t, gamma
from numpy.linalg import cholesky


np.random.seed(200)

# Nombre d'observations
nombre_observations = 50

# Matrice de modèle simulée
X = np.column_stack([np.ones(nombre_observations), np.random.normal(0, 1, nombre_observations),
                     np.random.normal(5, 10, nombre_observations), np.random.normal(100, 10, nombre_observations)])

# Vrais coefficients beta
vrais_coefficients_beta = np.array([1000, 50, -50, 10])

# Vraie valeur de phi
vraie_phi = 10000
matrice_identite = np.eye(nombre_observations)  # Matrice identité utilisée pour la matrice de covariance

# Simuler la variable dépendante pour la régression
y = multivariate_normal.rvs(mean=np.dot(X, vrais_coefficients_beta),
                            cov=vraie_phi * matrice_identite)


# On fait une fonction d'échantillonnage pour chaque paramètre : 

In [ ]:
def sample_eta(beta, sigma_sq, zeta, nu):
    """
    Échantillonne les valeurs de eta_t+1 conditionnellement à beta, sigma_sq, zeta, et nu.

    Arguments :
    beta : Vecteur de coefficients beta_t.
    sigma_sq : Variance sigma_t^2.
    zeta : Valeur zeta_t.
    nu : Paramètre nu.

    Returns :
    eta_sampled : Valeurs échantillonnées de eta_t+1.
    """
    
    eta_sampled = np.zeros(p)

    for j in range(p):
        m_tj = zeta * beta[j]**2 / (2 * sigma_sq)
        # Calcul du terme de pondération
        weight = np.exp(-m_tj * eta_sampled[j])
        # Calcul du terme de normalisation
        normalization = eta_sampled[j]**((1 - nu) / 2) * (1 + nu * eta_sampled[j])**(nu + 1)
        # Échantillonnage de eta_t+1,j
        eta_sampled[j] = np.random.gamma(shape=(1 - nu) / 2, scale=1 / normalization) * weight
    return eta_sampled





In [ ]:
# il faut faire une fonction calculate_likelihood ici 

def sample_zeta(zeta_previous, eta_sampled):
    """
    Échantillonne la valeur de zeta_t+1 conditionnellement à zeta_previous et eta_sampled.

    Arguments :
    zeta_previous : Valeur zeta_t.
    eta_sampled : Valeurs échantillonnées de eta_t+1.

    Returns :
    zeta_sampled : Valeur échantillonnée de zeta_t+1.
    """
    
    # Prior pris en zeta_t+1
    log_prior_zeta = -0.5 * (zeta_previous ** 2)
    
    # On sample une nouvel échantillon pour zeta_t+1 
    zeta_proposed = np.random.normal(np.log(zeta_previous), np.sqrt(0.5), size=1)
    
    # Prior sur le zeta_proposé
    log_prior_zeta_proposed = -0.5 * (zeta_proposed ** 2)
    
    # On calcule la vraisemblance conditionnelle des données
    log_likelihood = calculate_likelihood(y, zeta_proposed, eta_sampled) # cf fonction calculate_likelihood au dessus
    
    # On calcule le posterior pour l'ancien échantillon de zeta et le nouvel échantillon de zeta
    log_posterior_old = log_likelihood + log_prior_zeta       # "+" car logarithme
    log_posterior_proposed = log_likelihood + log_prior_zeta_proposed
    
    # Calculer le ratio d'acceptation
    acceptance_ratio = np.exp(log_posterior_proposed - log_posterior_old)
    
    # On accepte ou on rejete le nouvel échantillon de zeta
    if np.random.uniform(0, 1) < acceptance_ratio:
        zeta_sampled = np.exp(zeta_proposed)
    else:
        zeta_sampled = zeta_previous

    return zeta_sampled

In [ ]:
import numpy as np

def sample_zeta(zeta_previous, sigma_mrth):
    """
    Échantillonne la valeur de zeta_t+1 conditionnellement à zeta_previous.

    Arguments :
    zeta_previous : Valeur zeta_t.
    sigma_mrth : Écart-type de la proposition normale.

    Returns :
    zeta_sampled : Valeur échantillonnée de zeta_t+1.
    """
    # Proposition d'un nouvel échantillon de log(zeta_t+1)
    log_zeta_proposed = np.random.normal(np.log(zeta_previous), sigma_mrth)
    # Calcul des termes de probabilité a priori
    prior_current = -0.5 * zeta_previous**2
    prior_proposed = -0.5 * np.exp(2*log_zeta_proposed)
    # Calcul des termes de vraisemblance (non implémentés ici)
    # Log-probabilité du log-posterior pour les valeurs actuelles et proposées
    log_posterior_current = log_likelihood_current + prior_current
    log_posterior_proposed = log_likelihood_proposed + prior_proposed
    # Calcul du ratio de probabilité
    acceptance_ratio = np.exp(log_posterior_proposed - log_posterior_current)
    # Acceptation ou rejet de la proposition
    if np.random.uniform(0, 1) < acceptance_ratio:
        zeta_sampled = np.exp(log_zeta_proposed)
    else:
        zeta_sampled = zeta_previous

    return zeta_sampled


In [ ]:
def sample_sigma_sq(y, X, beta, zeta, a0, b0):
    """
    Échantillonne la variance sigma_sq_t+1 conditionnellement à y, X, beta, zeta, a0, et b0.

    Arguments :
    y : Vecteur de réponse.
    X : Matrice de prédicteurs.
    beta : Vecteur de coefficients beta_t.
    zeta : Valeur zeta_t.
    a0 : Paramètre a0.
    b0 : Paramètre b0.

    Returns :
    sigma_sq_sampled : Valeur échantillonnée de sigma_sq_t+1.
    """
    n = len(y)
    a = 0.5 * (n + a0)
    b = 0.5 * (np.dot(y.T, y) + b0)
    sigma_sq_sampled = invgamma.rvs(a, scale=b)

    return sigma_sq_sampled